In [1]:
directoryN = "/ipynbmorris/"
import sys; 
CPBdir = "../../../../cpb3101/CPlantBox"
sys.path.append(CPBdir+"/src");
sys.path.append(CPBdir);
sys.path.append("../../..");sys.path.append(".."); 
sys.path.append(CPBdir+"/src/python_modules");
sys.path.append("../build-cmake/cpp/python_binding/") # dumux python binding
sys.path.append("../../build-cmake/cpp/python_binding/")
sys.path.append("../modules/") # python wrappers 
import numpy as np
import json
import plantbox as pb
import importlib
importlib.reload(pb)
import plantbox
importlib.reload(plantbox)
from phloem_flux import PhloemFluxPython 
import pickle
from SALib.sample import saltelli
from SALib.analyze import sobol
from SALib.test_functions import Ishigami
from joblib import Parallel, delayed
import os 
from helpUqr import *

In [2]:
simDuration = 18;condition="dry"
weatherX = weather(simDuration, condition, 1)
assert weatherX['Qlight']>0

#			default value of env variable, to re-set at runtime
cs = 390e-6; 

Chl = 55,; 
oi = 210e-3;#leaf internal [O2] [mol mol-1]

SPAD= 100.0
chl_ = (0.114 *(SPAD**2)+ 7.39 *SPAD+ 10.6)/10


Q10 = 2.; #double TrefQ10 = 20;//to compute effect of T on growth (see CN-wheat, residual respiration @Barillot 2016, appendix)
#psiMax = 0;  
psiMin = -2000*(1/0.9806806);#limit wat. pot. in xylem for water-limited growth, [cm]
KMfu = 0.2; #@see C_fluxes,Michaelis menten coef for active sucrose usage
Vmaxloading = 0.019872;#mmol cm-1 d-1 for leaf blade 1cm wide
CSTimin = 0.4;#minimum CST value below which there is no sink of sucrose
beta_loading = 1;#@see C_fluxes, feedback effect of C_ST on Q_FL
Mloading = 0.2;#@see C_fluxes,Michaelis menten coef for Fl
Gr_Y = 0.75;#growth efficiency
psi_osmo = -10000*1.0197
delta_osmo_min = -psi_osmo - psiMin



namesVars_phlo = ['Q10','delta_osmo_min','psiMin','KMfu',
                  'Vmaxloading','CSTimin','beta_loading', 
                  'Mloading','Gr_Y','rhoSucrose',
             'kx_st','kr_st','Across','krm2','krm1','rmax']
namesLim_phlo = [[Q10*0.5, Q10*2],[delta_osmo_min/2,delta_osmo_min*2],[psiMin*2,psiMin/2],[KMfu*0.5,KMfu*2],
                 [Vmaxloading*0.5,Vmaxloading*2],[CSTimin*0.5,CSTimin*2],[0,1],# [beta_loading*0.5, beta_loading*2], 
                 [Mloading/2,Mloading*2],
             [Gr_Y*0.5,Gr_Y*2],[0.5,2],
             [0.5,2],[0.5,2],[0.5,2],[0.5,2],[0.5,2],[0.5,2] ]



namesVars_xyl = ['Chl','oi','fw1r','k_fw1','psi_t,crit,1',
              'kchl1', 'kchl2',
             'kg1','kjmax', 'alpha','omega',
             'gamma0', 'gamma1', 'gamma2','g0',
             'kx_x','kr_x']
namesLim_xyl = [1 for ii in namesVars_xyl]
problem = {
    'num_vars': len(namesVars_phlo),
    'names': namesVars_phlo,
    'bounds': namesLim_phlo
}
namesVars = namesVars_xyl + namesVars_phlo

##### TO CHANGE ######
Nreps = 11
if (simDuration > 10):# and (condition == "dry"):
    Nreps = 8
reps =2**Nreps
maxcore =  os.cpu_count()
print('nreps, cpu',Nreps, maxcore)
######################
param_values_phlo = saltelli.sample(problem, reps)

Yexuds = np.array([])
Ygrs  = np.array([])
Yrms = np.array([])

didDo=0
i=0

nreps, cpu 8 32


/tmp/ipykernel_482271/2550541080.py:62: DeprecationWarning: `salib.sample.saltelli` will be removed in SALib 1.5. Please use `salib.sample.sobol`
  param_values_phlo = saltelli.sample(problem, reps)


In [12]:
print(list(namesLim_xyl)+ list(param_values_phlo[didDo + i]))

[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1.017578125, 15259.651151986587, -2005.5427577541554, 0.1884765625, 0.03316528125, 0.264453125, 0.365234375, 0.20722656250000002, 0.697998046875, 1.1123046875, 1.1826171875, 1.9970703125, 0.8544921875, 0.9833984375, 1.2705078125, 0.6904296875]


In [ ]:
SobolTest(i,weatherX, 
         namesVars, #all the var names
         list(namesLim_xyl)+ list(param_values_phlo[didDo + i]), #all the var values
         simDuration,condition,"Phloem")

In [3]:
myid = i; varNames=namesVars; varLims=list(namesLim_xyl)+ list(param_values_phlo[didDo + i]);  testType="Phloem"
assert testType=="Xylem" or testType == "Phloem"
test_values = varLims
DictVal = {}
for key in varNames:
    for value in test_values:
        DictVal[key] = value
        test_values.remove(value)
        break

pl = pb.MappedPlant(seednum = 2) 
path = CPBdir+"/modelparameter/plant/"
name = "Triticum_aestivum_test_2021"

pl.readParameters(path + name + ".xml")
depth = 60
sdf = pb.SDF_PlantBox(np.Inf, np.Inf, depth )
pl.setGeometry(sdf) # creates soil space to stop roots from growing out of the soil
pl.initialize(verbose = True)#, stochastic = False)
pl.simulate(simDuration, False)#, "outputpm15.txt")

picker = lambda x,y,z : max(int(np.floor(-z)),-1)   
pl.setSoilGrid(picker)  # maps segment


r = PhloemFluxPython(pl,psiXylInit =-1000,ciInit = 0.5 *weatherX['cs'])


hp = max([tempnode[2] for tempnode in r.get_nodes()]) /100
weatherX = weather(simDuration, condition, hp)

plant =setDefaultVals(r,weatherX,DictVal)

#xXx = np.full((len(r.plant.nodes)),weatherX["p_mean"])
p_bot = weatherX["p_mean"] + depth/2
p_top = weatherX["p_mean"] - depth/2
xXx = np.linspace(p_top, p_bot, depth)

plant.Q10 = DictVal['Q10']# X[0]
plant.psi_osmo_proto= -DictVal['delta_osmo_min'] - DictVal['psiMin']
plant.psiMin = DictVal['psiMin']
plant.KMfu = DictVal['KMfu']
plant.Vmaxloading = DictVal['Vmaxloading']
plant.CSTimin = DictVal['CSTimin']
plant.beta_loading = DictVal['beta_loading']
plant.Mloading = DictVal['Mloading']
plant.Gr_Y= DictVal['Gr_Y']# X[8]

plant.Patm = weatherX["Pair"]
##resistances
plant.g_bl = resistance2conductance(weatherX["rbl"],weatherX,r) / r.a2_bl
plant.g_canopy = resistance2conductance(weatherX["rcanopy"],weatherX,r) / r.a2_canopy
plant.g_air = resistance2conductance(weatherX["rair"],weatherX,r) / r.a2_air
plant.gm = 0.025
#plant.fwr = 0
#r.sh = 4e-4

plant.usePg4Fw = True
#r.doWiggle= True;r.DoSun2012=False
plant.alternativeAn = False
plant.Theta_ag = 0.9
plant.limMaxErr = 1/100
plant.maxLoop = 10000

verbose_phloem = True
filename = "sobolPhoto4/inPM"+repr(myid)+repr(simDuration)+repr(testType)+".txt"
#plant.doTroubleshooting=True
if testType == "Phloem":
    directoryN = "/sobolST/"
else:
    directoryN = "/sobolX/"
try:

    plant.solve_photosynthesis(sim_time_ = simDuration, sxx_=xXx, 
                               cells_ = True,ea_ = weatherX["ea"],es_ = weatherX["es"],
            verbose_ = False, doLog_ = False,TairC_= weatherX["TairC"] ,outputDir_= "./results"+directoryN)



    if testType == "Phloem":
        plant.startPM(simDuration, simDuration + 2/24, 1, ( weatherX["TairC"]  +273.15) , verbose_phloem, filename)
        Nt = len(plant.plant.nodes) 
        Q_Rm    = np.array(plant.Q_out[(Nt*2):(Nt*3)])
        Q_Exud  = np.array(plant.Q_out[(Nt*3):(Nt*4)])
        Q_Gr    = np.array(plant.Q_out[(Nt*4):(Nt*5)])
    elif testType == "Xylem":
        Ev = sum(plant.Ev)#*1/(24)
        Ag = np.sum(plant.Ag4Phloem)#*1/(24)
except Exception as e:
    name2 ="./results"+directoryN+'IDError.txt'
    with open(name2, 'a') as log:
        log.write(repr(simDuration)+" "+repr( myid) +" "+str(e) +'\n')
    plant.doTroubleshooting = True
    plant.solve_photosynthesis(sim_time_ = 21, sxx_=xXx, 
                               cells_ = True,ea_ = weatherX["ea"],es_ = weatherX["es"],
            verbose_ = False, doLog_ = False,TairC_= weatherX["TairC"] )
    if False:#testType == "Phloem":
        plant.startPM(simDuration, simDuration + 1/24, 1, ( weatherX["TairC"]  +273.15) , verbose_phloem, filename)
        Nt = len(plant.plant.nodes) 
        Q_Rm    = np.array(plant.Q_out[(Nt*2):(Nt*3)])
        Q_Exud  = np.array(plant.Q_out[(Nt*3):(Nt*4)])
        Q_Gr    = np.array(plant.Q_out[(Nt*4):(Nt*5)])
    raise Exception("error at my id "+repr(myid))

try:
    os.remove(filename)
except OSError:
    pass


MappedPlant::initialize 
Seed::initialize: Plant 
Seed::initialize: Basal root type #4 was not defined, using tap root parameters instead
Tiller stem type #4 was not defined, using main stem parameters instead, Exchange zone in roots: kr > 0 until 0.8cm from root tip
Kr is constant per subtype of organ type, for root, subtype 0 = 6.49549e-05 1 day-1 
Kx is constant per subtype of organ type, for root, subtype 0 = 0.0880999 cm3 day-1 
Exchange zone in roots: kr_st > 0 until 0.8cm from root tip 1 1
Exchange zone in roots: kr > 0 until 0.8cm from root tip
Kr_st is constant per subtype of organ type, for root, subtype 1 = 0.0998535 1 day-1 
Kx_st is constant per subtype of organ type, for root, subtype 1 = 1.56805e-13 cm3 day-1 
Across_st is constant per subtype of organ type, for root, subtype 1 = 8.22118e-06 cm2 
krm2 is constant 1.9668e-05 -
krm1 is constant 0.127051 -
rhoSucrose is constant per organ type, organ type 2 (root) = 0.567275 mmol cm-3
Rmax_st is constant per subtype of orga

In [9]:
np.sum(plant.Ag4Phloem)

0.9845248368010105

In [6]:
np.array(Q_Rm)

array([0.00000000e+00, 2.36542187e-06, 3.45229899e-07, 5.11494610e-06,
       5.11492706e-06, 5.11495244e-06, 5.11495225e-06, 3.51249410e-06,
       3.30310904e-07, 3.23683121e-07, 3.17062377e-07, 5.14509667e-06,
       3.45228994e-07, 5.11493703e-06, 5.11495291e-06, 5.11494861e-06,
       5.11495034e-06, 3.51249349e-06, 3.30309998e-07, 3.23682220e-07,
       3.17061482e-07, 5.14506025e-06, 3.45229823e-07, 5.11494514e-06,
       5.11494831e-06, 5.11494321e-06, 5.11495268e-06, 3.51249372e-06,
       3.30309999e-07, 3.23681798e-07, 3.17060636e-07, 5.14495425e-06,
       2.13023901e-06, 3.08854516e-07, 4.40528936e-06, 4.40526765e-06,
       4.40528998e-06, 4.40528573e-06, 3.02222501e-07, 2.95597059e-07,
       2.88977653e-07, 2.82363978e-07, 4.76709514e-06, 3.08855430e-07,
       4.40531248e-06, 4.40530899e-06, 4.40530836e-06, 4.40530962e-06,
       3.02224890e-07, 2.95600930e-07, 2.88983011e-07, 2.82370831e-07,
       4.76758520e-06, 3.08852668e-07, 4.40528446e-06, 4.40530306e-06,
      